In [ ]:
"""
sbm_experiments.ipynb

Created on Sat Nov 26 09:38:00 2022

@author: Lukas

This script contains all experiments conducted with stochastic block models.
"""

In [ ]:
# load packages using the setup.py script

from setup import *

from compute_curvatures import *

In [ ]:
# define auxilliary functions for SBMs

def build_size_list (k, l):
    ll = [k  for i in range(l)]
    return ll


def build_prob_list (n, p_in, p_out):
    ll = []
    for i in range(n):    
        temp_l = [p_out  for j in range(0,i)] + [p_in] + [p_out  for j in range(i+2,n+1)]
        ll.append(temp_l)
    return ll

In [ ]:
def calculate_SBM(k, l, p_in, p_out, title_str, t_coeff, q_coeff, p_coeff):
    """
    Calculates the curvatures of a SBM with k nodes in each of l blocks.
    
    Parameters
    ----------
    k : int
        number of nodes in each block

    l : int
        number of blocks

    p_in : float
        probability of an edge within a block

    p_out : float
        probability of an edge between blocks

    title_str : str
        title of the plot

    t_coeff : float
        coefficient for the triangle term in the augmented Forman-Ricci curvature

    q_coeff : float
        coefficient for the quadrilateral term in the augmented Forman-Ricci curvature

    p_coeff : float
        coefficient for the pentagon term in the augmented Forman-Ricci curvature

    Returns
    -------
    None.
    
    """
    print("k:",k," l:",l," p_in:",p_in," p_out:",p_out," t_coeff:",t_coeff," q_coeff:",q_coeff," p_coeff:",p_coeff)
    sizes = build_size_list(k, l)
    probs = build_prob_list(l, p_in, p_out)
    
    G = nx.stochastic_block_model(sizes, probs, seed = 0)
    init_edge_attributes(G)
      
    H = G.to_directed()
    
    # pos1 = nx.kamada_kawai_layout(H)
    # blocks = [v["block"]  for u,v in H.nodes.data()]
    # plot_my_graph(H, pos1, node_col = blocks)
    
    t0 = perf_counter()
    cycles = []
    for c in simple_cycles(H, 6):
        cycles.append(c) 
    t1 = perf_counter()
    print("Zyklen: ",len(cycles), " - Zeit: ", t1-t0)
    
    d = dict()
    for i in range(3,6):
        d[i] = [c  for c in cycles  if len(c) == i]
        
        # d[i] = remove_permutations(d[i])
        
        # t2 = perf_counter()
        # print(i, " ohne Permutationen - Zeit: ", t2-t1, " / ", end = "")
        set_edge_attributes_2(G, d[i], i)
        # t3 = perf_counter()
        # print(i, "Edge Attributes - Zeit: ", t3-t2)
        # t1 = t3
        
    get_orc_edge_curvatures (G)
    get_edge_curvatures (G, t_coeff, q_coeff, p_coeff)
    # t4 = perf_counter()
    # print("Curvature - Zeit: ", t4-t3)
    
    show_curv_data (G, title_str)
    # t5 = perf_counter()
    # print("Histos - Zeit: ", t5-t4)

In [ ]:
def calculate_SBMs():
    """
    Calculates the curvatures of a SBM with k nodes in each of l blocks.

    Parameters
    ----------
    None.

    Returns
    -------
    None.
    """

    ll_k = [5,10,15,20]
    k_def = 20
    ll_l = [2,3,4,5]
    l_def = 5
    ll_p_in = [0.6, 0.7, 0.8, 0.9]
    p_in_def = 0.7
    ll_p_out = [0.05, 0.03, 0.02, 0.01]
    p_out_def = 0.05

    for k in ll_k:
        s = "Variation of community size / k = " + str(k) + "\n" + \
            "k=" + str(k) + " l=" + str(l_def) + " p_in:" + str(p_in_def) + " p_out:" + str(p_out_def)
        calculate_SBM(k, l_def, p_in_def, p_out_def, s, 3, 2, 1)

    for l in ll_l:
        s = "Variation of number of communities / l = " + str(l) + "\n" + \
            "k=" + str(k_def) + "  l=" + str(l) +  "  p_in=" + str(p_in_def) + "  p_out=" + str(p_out_def)
        calculate_SBM(k_def, l, p_in_def, p_out_def, s, 3, 2, 1)

    for p_in in ll_p_in:
        s = "Variation of p_in / p_in = " + str(p_in) + "\n" + \
            "k=" + str(k_def) + " l=" + str(l_def) +  " p_in:" + str(p_in) + " p_out:" + str(p_out_def)
        calculate_SBM(k_def, l_def, p_in, p_out_def, s, 3, 2, 1)
        
    for p_out in ll_p_out:
        s = "Variation of p_out / p_out = " + str(p_out) + "\n" + \
            "k=" + str(k_def) + " l=" + str(l_def) +  " p_in:" + str(p_in_def) + " p_out:" + str(p_out)
        calculate_SBM(k_def, l_def, p_in_def, p_out, s, 3, 2, 1)